# RAG and prompt automation in dspy

In this notebook, we implement RAG pipeline and RAG compiled with prompt automation with DSPy on Amazon Bedrock. 

### Environment Setup

Uncomment the following lines if the packages are not installed 

In [1]:
%%capture
!pip3 install langchain
!pip3 install -U langchain-community
!pip3 install pypdf
!pip3 install chromadb
!pip3 install dspy
!pip3 install dspy-ai[faiss-cpu]

In [2]:
import json, os, sys
import io, base64
import pandas as pd
import pathlib
from PIL import Image as PILImage
import unicodedata
from IPython.display import Markdown
import boto3
import dspy
from dspy.teleprompt import BootstrapFewShot
import pandas as pd
import logging
import json, os, shutil
import boto3, re, os, shutil
import matplotlib.pyplot as plt
    

Environment setup : clean up temporary trace files and data files , set up loggers, bedrock session

In [3]:
def clean_up_files_in_dir(dir):
    # cleanup trace files to avoid issues
    if os.path.isdir(dir):
        shutil.rmtree(dir)
    os.mkdir(dir)

In [4]:
clean_up_files_in_dir('./cuad_db')
clean_up_files_in_dir('./trace_files')

In [5]:
# setting logging
logging.basicConfig(filename='lab2_trace.log', 
                    filemode='a', 
                    encoding='utf-8',
                    format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', 
                    level=logging.INFO)

In [6]:
boto_session = boto3.session.Session()
region_name = boto_session.region_name
print(region_name)

session = boto3.Session()

bedrock = session.client("bedrock", region_name=region_name)
br = session.client("bedrock-runtime", region_name=region_name)



us-east-1


### DSPy Language Model and Retriever Model configuration

In [7]:
claude_sonnet_model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
clade_haiku_model_id = "anthropic.claude-3-haiku-20240307-v1:0"
titan_embed_model_id = "amazon.titan-embed-text-v2:0"
mistral_7b_model_id = "mistral.mistral-7b-instruct-v0:2"
llama_8b_model_id = "meta.llama3-8b-instruct-v1:0"

##### DSPy Initialization requires you to explicitly set the reion

In [8]:
dsp_bedrock = dspy.Bedrock(region_name=region_name)

bedrock_sonnet = dspy.AWSAnthropic(aws_provider=dsp_bedrock,
                                  model=claude_sonnet_model_id,
                                  max_new_tokens=4096,
                                  max_tokens=4096)

bedrock_mistral_7b = dspy.AWSMistral(aws_provider=dsp_bedrock,
                                 model=mistral_7b_model_id,
                                 max_new_tokens=4096,
                                 max_tokens=4096)

bedrock_haiku = dspy.AWSAnthropic(aws_provider=dsp_bedrock,
                                 model=clade_haiku_model_id,
                                 max_new_tokens=4096,
                                 max_tokens=4096)

bedrock_llama_8b = dspy.AWSMeta(aws_provider=dsp_bedrock, 
                                model=llama_8b_model_id,
                                max_new_tokens=2048,
                                max_tokens=2048)

### Splitters and chunking configuration    

We will be using the RecursiveCharacterTextSplitter to generate logical and syntactically readable chunks. The size and overlap percentage can be empirically determined based on the dataset. For more flexibility it is possible to generate multiple files from the dataset file and make 1 file 1 chunk.

In [9]:
from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader, CSVLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

# Load documents 
loader = PyPDFLoader("../cuad_data/CUAD_v1/full_contract_pdf/Part_I/Strategic Alliance//ENERGOUSCORP_03_16_2017-EX-10.24-STRATEGIC ALLIANCE AGREEMENT.PDF")
documents = loader.load()

# Split documents by setting chunk size
text_splitter = RecursiveCharacterTextSplitter(
    separators=[
        "\n\n",
        "\n",
        " ",
        ".",
        ",",
        "\u200b",  # Zero-width space
        "\uff0c",  # Fullwidth comma
        "\u3001",  # Ideographic comma
        "\uff0e",  # Fullwidth full stop
        "\u3002",  # Ideographic full stop
        "",
    ],
    chunk_size = 5000,      
    chunk_overlap = 30, 
    length_function=len,
)

docs = text_splitter.split_documents(documents)

### Clean and remove any empty pages in the PDF

In [10]:
print(len(docs))
clean_docs = []
for doc in docs:
    if len(doc.page_content):
        clean_docs.append(doc)

docs = clean_docs
print(len(docs))

32
32


### Setup Retriever on local disk using ChromaDB    

We are using chromaDB to demonstrate the use of vector databases. 

In [11]:
import chromadb
from chromadb.utils import embedding_functions
from chromadb.utils.embedding_functions import AmazonBedrockEmbeddingFunction

bedrock_ef = AmazonBedrockEmbeddingFunction(session=session, model_name=titan_embed_model_id)

bedrock_embeddings = bedrock_ef([doc.page_content for doc in docs])

In [12]:
from dspy.retrieve.chromadb_rm import ChromadbRM

collection_name="contexts"
persist_dir="cuad_db/"

chroma_client = chromadb.PersistentClient(persist_dir)
collection = chroma_client.get_or_create_collection(name=collection_name)

if collection.count() == 0:
    collection.add(embeddings=bedrock_embeddings,
                   documents=[doc.page_content for doc in docs],
                   ids=[str(i) for i in range(len(docs))])

In [13]:
rm = ChromadbRM(collection_name=collection_name, 
                persist_directory=persist_dir, 
                embedding_function=bedrock_ef,
                k=3)

### Dataset setup in DSPy


Contract Understanding Atticus Dataset (CUAD) is a dataset for legal contract review. CUAD was created with dozens of legal experts from The Atticus Project and consists of over 13,000 annotations.


In [14]:
TRN_FILE = '../cuad_data/ENERGOUSCORP_qa.csv'
df_cuad_data = pd.read_csv(TRN_FILE)
df_cuad_data.head()


,index,question,input,answer,qa_id
0,0,What is The name of the contract?,Highlight the parts (if any) of this contract ...,STRATEGIC ALLIANCE AGREEMENT,ENERGOUSCORP_03_16_2017-EX-10.24-STRATEGIC ALL...
1,1,What is The two or more parties who signed the...,Highlight the parts (if any) of this contract ...,"Dialog Semiconductor (UK) Ltd., DIALOG, Energo...",ENERGOUSCORP_03_16_2017-EX-10.24-STRATEGIC ALL...
2,2,What is The date of the contract?,Highlight the parts (if any) of this contract ...,"November 6, 2016",ENERGOUSCORP_03_16_2017-EX-10.24-STRATEGIC ALL...
3,3,What is The date when the contract is effective?,Highlight the parts (if any) of this contract ...,"November 6, 2016",ENERGOUSCORP_03_16_2017-EX-10.24-STRATEGIC ALL...
4,4,On what date will the contract's initial term ...,Highlight the parts (if any) of this contract ...,"Unless earlier terminated as provided herein, ...",ENERGOUSCORP_03_16_2017-EX-10.24-STRATEGIC ALL...


In [15]:
dataset_cuad = []

for index, question, input, answer, qa_id in df_cuad_data.values:
    dataset_cuad.append(dspy.Example(question=question, answer=answer).with_inputs("question"))

trainset = [x.with_inputs('question') for x in dataset_cuad[0:28]]
devset = [x.with_inputs('question') for x in dataset_cuad[28:32]]

### Factuality Judge and Metrics setup (LLM as a judge)

We will now create a Factuality Judge that will adjudicate if the predicted answer factually correct to the groundtruth answer and means the same as the groundtruth answer.

A new factuality metric is created to mark whether the predicted answer is True or not.

In [16]:
class FactJudge(dspy.Signature):
    """Judge if the predicted answer factually correct to the groundtruth answer and same as groundtruth answer. Answer either Factual[True] or Factual[False]"""

    question = dspy.InputField(desc="Question to be answered")
    groundtruth_answer = dspy.InputField(desc="groundtruth answer for the question")
    predicted_answer = dspy.InputField(desc="predicted answer for the question")
    factually_correct = dspy.OutputField(desc="Is the predicted answer factually correct to the groundtruth answer and same as groundtruth answer ?", prefix="Factual[True/False]:")


judge = dspy.ChainOfThought(FactJudge)


def factuality_metric(example, pred):
    factual = judge(question=example.question, groundtruth_answer=example.answer, predicted_answer=pred.answer) #context=pred.context, 
    #logging.info(f"\n factual LLM judge {factual}")
    #logging.info(f"\n example.answer {example.answer}")
    #logging.info(f"\n pred.answer {pred.answer}")
    llm_judge_ans = bool("Factual[True]" in factual.factually_correct 
                         or '100% True' in factual.factually_correct
                         or '100% factually correct' in factual.factually_correct
                         or factual.factually_correct=='True') #or "correct" in factual.factually_correct.lower()
    print(f"llm_judge_ans = {llm_judge_ans}")
    logging.info(f"llm_judge_ans = {llm_judge_ans}")
    return llm_judge_ans

### LLM as a judge evaluation

Let us setup the metric, dataset and RAG pipeline with input and output fields. Then we customize how to validate where we can combine multiple metrics to come up with the final output. 

In [17]:
from dspy.evaluate import Evaluate

# Evaluation
metric_LLM = factuality_metric

evaluate_cuad = Evaluate(devset=dataset_cuad, metric=metric_LLM, num_threads=1, display_progress=True, display_table=0)

### Setup RAG pipeline

In [18]:
# RAG pipeline

class GenerateAnswer(dspy.Signature):
    """Answer questions with short factoid answers."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

In [19]:
class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()
        self.retrieve = dspy.Retrieve(k=num_passages)
        #self.retrieve = ChromadbRM("contexts", "./chroma", k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
    
    def forward(self, question):
        #context = self.retrieve(question).passages
        context = [unicodedata.normalize("NFKD", r) for r in self.retrieve(question).passages]
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)

In [20]:
def validate_context_and_answer(example, pred, trace=None):
        answer_EM = dspy.evaluate.answer_exact_match(example, pred)
        answer_PM = dspy.evaluate.answer_passage_match(example, pred)
        answer_LLMJudge = factuality_metric(example, pred)

        logging.info(f"\n example question :: {example.question} , example answer :: {example.answer} ")
        logging.info(f"\n pred answer :: {pred.answer}")
        logging.info(f"\n answer_EM :: {answer_EM}, answer_PM ::{answer_PM} answer_LLMJudge :: {answer_LLMJudge}")
        return answer_LLMJudge or answer_EM or answer_PM


### Raw Model Evaluation and Compiled Model Evaluation

We will compare the performance against the raw and compiled DSPy network in this section with different model families and sub-types. Here we try out Haiku, Sonnet and Mistral 7B. This will take some time to run.

In [21]:
def generate_csv(rag_lm, model_id):
    # Loop and generate CSV

    csv_list = list()
    for index, question, input, answer, qa_id in df_cuad_data.values:
        # Get the prediction. This contains `pred.context` and `pred.answer`.
        pred = rag_lm(question)
        csv_list.append([question, answer, pred.answer])

    df = pd.DataFrame(csv_list, columns=['question', 'gt_answer', 'pred_answer'])
    df.to_csv(f"RAG_{model_id}.csv", header=True)

In [ ]:
%%time
# Evaluation

raw_scores = list()
compiled_scores = list()
model_ids = ["anthropic.claude-3-haiku-20240307-v1:0", "anthropic.claude-3-sonnet-20240229-v1:0", "mistral.mistral-7b-instruct-v0:2"]
counter = 0

for lm_id in [bedrock_haiku, bedrock_sonnet, bedrock_mistral_7b]:

    dspy.settings.configure(lm=lm_id)
    dspy.settings.configure(rm=rm)
    rag_lm = RAG()
    print(f"START OF model_id = {model_ids[counter]} --------------->")
    raw_score = evaluate_cuad(rag_lm, num_threads=1)
    raw_scores.append(raw_score)
    
    print(f"raw_score for lm_id = {model_ids[counter]} and raw_score =  {raw_score}")
    logging.info(f"raw_score for lm_id = {model_ids[counter]} and raw_score =  {raw_score}")
    
    # Validation logic: check that the predicted answer is correct.
    # Also check that the retrieved context does actually contain that answer.
    
    # Set up a basic teleprompter, which will compile our RAG program.
    teleprompter = BootstrapFewShot(metric=validate_context_and_answer)
    
    print(f"******* COMPILING model_id = {model_ids[counter]} ******* --------------->")
    # Compile!
    compiled_rag = teleprompter.compile(rag_lm, trainset=trainset)
    
    compiled_score = evaluate_cuad(compiled_rag, num_threads=1)
    compiled_scores.append(compiled_score)
    
    print(f"compiled_score for COMPILED lm_id = {model_ids[counter]} and raw_score =  {compiled_score}")
    logging.info(f"compiled_score for COMPILED lm_id = {model_ids[counter]} and raw_score =  {compiled_score}")
    
    lm_id.inspect_history(n=1)
    generate_csv(rag_lm, model_ids[counter])
    print(f"END OF model_id = {model_ids[counter]} --------------->")
    counter +=1
    

START OF model_id = anthropic.claude-3-haiku-20240307-v1:0 --------------->
  0%|          | 0/32 [00:00<?, ?it/s]

	*** In DSPy 2.5, all LM clients except `dspy.LM` are deprecated, underperform, and are about to be deleted. ***
 		You are using the client AWSAnthropic, which will be removed in DSPy 2.6.
 		Changing the client is straightforward and will let you use new features (Adapters) that improve the consistency of LM outputs, especially when using chat LMs. 

 		Learn more about the changes and how to migrate at
 		https://github.com/stanfordnlp/dspy/blob/main/examples/migration.ipynb


llm_judge_ans = False
  0%|          | 0/32 [00:02<?, ?it/s]

llm_judge_ans = False


Average Metric: 0.00 / 32 (0.0%):   3%|▎         | 1/32 [00:06<01:30,  2.93s/it]

...

Average Metric: 22.00 / 32 (68.8%):  88%|████████▊ | 28/32 [03:36<00:30,  7.72s/it]

llm_judge_ans = True


Average Metric: 23.00 / 32 (71.9%):  91%|█████████ | 29/32 [03:44<00:26,  8.73s/it]

llm_judge_ans = True


Average Metric: 24.00 / 32 (75.0%):  94%|█████████▍| 30/32 [03:44<00:17,  8.66s/it]

### Retriever Evaluation (RM is same across models)

In RAG workflows, it is very crucial to understand the retriever performance to be able to improve the performance of the overall system. We will now create a retriever judge to understand how well the retriever is able to pull out the relevant chunks for the incoming user question. Context and Answer relevance is measured using this Retriever Judge.

In [ ]:
class RetreivalJudge(dspy.Signature):
    """Judge given the question to be answered, check if the groundtruth answer can be derived from the predicted context.  Answer either Retrieved[True] or Retrieved[False]"""

    context = dspy.InputField(desc="Context for the prediction")
    question = dspy.InputField(desc="Question to be answered")
    groundtruth_answer = dspy.InputField(desc="groundtruth answer for the question")
    retrieval_correctness = dspy.OutputField(desc="Can the groundtruth answer be derived from the predicted context?", prefix="Retrieved[True/False]:")

retreival_judge = dspy.ChainOfThought(RetreivalJudge)


def retrieval_metric(example, pred):
    retrieval = retreival_judge(question=example.question, groundtruth_answer=example.answer, context=pred.context) 
    #logging.info(f"\n retrieval LLM judge {retrieval}")
    #logging.info(f"\n example {example}")
    #logging.info(f"\n pred {pred}")
    llm_retreiver_ans = bool("Retrieved[True]" in retrieval.retrieval_correctness 
                         or '100% True' in retrieval.retrieval_correctness
                         or '100% retrieved correct' in retrieval.retrieval_correctness
                         or 'True.' in retrieval.retrieval_correctness) 
    #logging.info(f"llm_retreiver_ans = {llm_retreiver_ans}")
    return llm_retreiver_ans

In [ ]:
def answer_passages_retrieved(example, pred, trace=None):
    gt_answer = example['answer']
    pred_context = set(map(dspy.evaluate.normalize_text, [c.split(' | ')[0] for c in pred.context]))
    logging.info(f"example['answer'] :: {example['answer']}")
    logging.info(f"gt_answer :: {gt_answer}")
    #logging.info(f"pred_context :: {pred_context}")
    return gt_answer.issubset(pred_context)

compiled_rag_retrieval_score = evaluate_cuad(compiled_rag, num_threads = 1, metric=retrieval_metric)
compiled_rag_retrieval_score

### RAG performance across models

Let us see the performance across various model familes and types using raw and compiled DSPy framework to understand what value DSPy may bring to the table. 

In [ ]:
class_type = ['RAG_Haiku','RAG_Haiku_Compiled','RAG_Sonnet','RAG_Sonnet_Compiled','RAG_Mistral','RAG_Mistral_Compiled']
class_count = [raw_scores[0],compiled_scores[0],raw_scores[1],compiled_scores[1],raw_scores[2],compiled_scores[2]]

fig = plt.figure(figsize = (10, 5))

# creating the bar plot
plt.bar(class_type, class_count, color ='green', width = 0.4)

for i in range(len(class_type)):
        plt.text(i,class_count[i],class_count[i], ha = 'center')

plt.xlabel("")
plt.ylabel("RAG Accuracy performance ")
plt.title("RAG performance across models")
plt.show()